In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0-rc0


In [3]:
"""
from IPython.core.display import display, HTML
from PIL import Image
from io import BytesIO
import base64
"""

'\nfrom IPython.core.display import display, HTML\nfrom PIL import Image\nfrom io import BytesIO\nimport base64\n'

In [4]:
main_folder = '../input/celeba-dataset/'
images_folder = main_folder + 'img_align_celeba/img_align_celeba/'

EXAMPLE_PIC = images_folder + '000506.jpg'

TRAINING_SAMPLES = 40000
VALIDATION_SAMPLES = 2000
TEST_SAMPLES = 2000
IMG_WIDTH = 178
IMG_HEIGHT = 218
BATCH_SIZE = 16
NUM_EPOCHS = 20

In [5]:
# import the data set that include the attribute for each picture
df_attr = pd.read_csv(main_folder + 'list_attr_celeba.csv')
df_attr.set_index('image_id', inplace=True)
df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
df_attr.shape

(202599, 40)

In [6]:
train_image = np.zeros((TRAINING_SAMPLES, 218, 178, 3), dtype=np.uint8)

for i in range(0, TRAINING_SAMPLES):
    train_image[i,:,:,:] = cv2.imread(images_folder + str(i+1).zfill(6) + ".jpg")/255

In [7]:
train_gd = np.zeros((TRAINING_SAMPLES, 1), dtype=np.uint8)

for i in range(0, TRAINING_SAMPLES):
    train_gd[i] = df_attr.loc[str(i+1).zfill(6)+".jpg"]['Smiling']

In [8]:
valid_image = np.zeros((VALIDATION_SAMPLES, 218, 178, 3), dtype=np.uint8)

for i in range(0, VALIDATION_SAMPLES):
    valid_image[i,:,:,:] = cv2.imread(images_folder + str(i+1+TRAINING_SAMPLES).zfill(6) + ".jpg")/255

valid_gd = np.zeros((VALIDATION_SAMPLES, 1), dtype=np.uint8)

for i in range(0, VALIDATION_SAMPLES):
    valid_gd[i] = df_attr.loc[str(i+1+TRAINING_SAMPLES).zfill(6)+".jpg"]['Smiling']

In [9]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(218, 178, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.SpatialDropout2D(0.1))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.SpatialDropout2D(0.1))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(8, activation='relu'))
#model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 216, 176, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 108, 88, 32)       0         
_________________________________________________________________
spatial_dropout2d (SpatialDr (None, 108, 88, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 106, 86, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 43, 64)        0         
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 53, 43, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 51, 41, 128)       7

In [10]:
model.input_shape

(None, 218, 178, 3)

In [11]:
model.fit(train_image, train_gd, validation_data=(valid_image, valid_gd), epochs=15)

Train on 40000 samples, validate on 2000 samples
Epoch 1/15
40000/40000 [==============================] - 39s 984us/sample - loss: 0.5717 - accuracy: 0.6753 - val_loss: 0.5476 - val_accuracy: 0.6900
Epoch 2/15
40000/40000 [==============================] - 31s 773us/sample - loss: 0.5090 - accuracy: 0.7165 - val_loss: 0.5481 - val_accuracy: 0.6850
Epoch 3/15
40000/40000 [==============================] - 31s 763us/sample - loss: 0.4425 - accuracy: 0.7580 - val_loss: 0.5963 - val_accuracy: 0.6660
Epoch 4/15
40000/40000 [==============================] - 30s 743us/sample - loss: 0.3521 - accuracy: 0.8106 - val_loss: 0.6664 - val_accuracy: 0.6520
Epoch 5/15
40000/40000 [==============================] - 30s 745us/sample - loss: 0.2637 - accuracy: 0.8586 - val_loss: 1.0135 - val_accuracy: 0.6575
Epoch 6/15
40000/40000 [==============================] - 30s 745us/sample - loss: 0.2086 - accuracy: 0.8835 - val_loss: 1.2052 - val_accuracy: 0.6560
Epoch 7/15
40000/40000 [=====================

In [ ]:
model.save("model.h5")